In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 검색어, 시작 날짜, 종료 날짜 입력
keywords = [
    "검색어",
]
start_date = "2024-06-01"
end_date = "2024-08-31"

collected_urls = set()  # 수집된 기사 URL을 저장하는 집합

driver = webdriver.Chrome(options=chrome_options)

# 기사 목록 및 "더 보기" 버튼을 통해 추가 기사를 로드
def fetch_search_results(keyword, start_date, end_date):
    previous_article_count = 0
    results = []
    base_url = "https://www.mk.co.kr/search?word="
    search_url = base_url + urllib.parse.quote(keyword) + f"&dateType=direct&startDate={start_date}&endDate={end_date}"
    driver.get(search_url)

    try:
        # 첫 번째 기사 목록이 뜰 때까지 기다림 (최대 10초)
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "li.news_node"))
        )
        print(f"'{keyword}' 기사 목록이 로드되었습니다.")
    except Exception as e:
        print(f"'{keyword}' 기사 목록을 로드하는 데 실패했습니다: {e}")
        return results  # 오류 시 빈 리스트 반환

    # 첫 번째로 로드된 기사 목록을 처리
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all("li", class_="news_node")
    for item in articles:
        try:

            link_tag = item.find("a", class_="news_item")
            url = link_tag["href"] if link_tag else ""
            url = "https:" + url if not url.startswith("http") else url

            if url and url not in collected_urls:
                collected_urls.add(url)  # 새로운 URL 저장

                title = item.find("h3", class_="news_ttl").get_text(strip=True) if item.find("h3", class_="news_ttl") else ""
                section = item.find("span", class_="cate").get_text(strip=True) if item.find("span", class_="cate") else ""

                date_tag = item.find("p", class_="time_info")
                date = date_tag.get_text(strip=True).split()[0] if date_tag else ""


                # 결과 리스트에 기사 추가
                results.append({
                    'media': '매일경제',
                    'title': title,
                    'date': date,
                    'section': section,
                    'article_type': '',
                    'writer': '',
                    'page': '',
                    'url': url,
                    'content': ''  # 본문 내용은 없음
                })
        except Exception as e:
            print(f"기사 처리 중 오류 발생: {e}")

    # "더 보기" 버튼 클릭 로직 시작
    while True:
        try:
            # "더 보기" 버튼을 찾고 클릭
            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.basic.w_f.drop_sub_news_btn.h_s.ic_before"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", load_more_button)  # 버튼까지 스크롤
                load_more_button.click()
                print(f"'{keyword}'에 대해 '더 보기' 버튼 클릭")
            except Exception:
                print(f"'{keyword}' 더 이상 '더 보기' 버튼이 없음.")
                break

            # 기사 목록이 추가적으로 로드될 때까지 대기 (최대 10초)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "news_node"))
            )

            # 추가된 기사 목록을 다시 파싱
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_articles = soup.find_all("li", class_="news_node")

            # 새로운 기사가 없으면 종료
            if len(new_articles) == previous_article_count:
                print(f"더 이상 기사가 로드되지 않음. 현재 기사 수: {len(new_articles)}")
                break
            previous_article_count = len(new_articles)

            # 새로 로드된 기사를 처리 (기사 정보 추출)
            for item in new_articles:  # 수정: new_articles를 처리하도록 변경
                try:
                    # 한 줄로 기사 정보 추출
                    url = "https:" + item.find("a", class_="news_item")["href"] if item.find("a", class_="news_item") and not item.find("a", class_="news_item")["href"].startswith("http") else item.find("a", class_="news_item")["href"]
                    title = item.find("h3", class_="news_ttl").get_text(strip=True) if item.find("h3", class_="news_ttl") else ""
                    section = item.find("span", class_="cate").get_text(strip=True) if item.find("span", class_="cate") else ""
                    date_tag = item.find("p", class_="time_info")
                    date = date_tag.get_text(strip=True).split()[0] if date_tag else ""

                    # 중복 URL 체크
                    if url and url not in collected_urls:
                        collected_urls.add(url)  # 새로운 URL 저장
                        # 결과 리스트에 기사 추가
                        results.append({
                            'media': '매일경제',
                            'title': title,
                            'date': date,
                            'section': section,
                            'article_type': '',
                            'writer': '',
                            'page': '',
                            'url': url,
                            'content': ''  # 본문 내용은 없음
                        })
                except Exception as e:
                    print(f"기사 처리 중 오류 발생: {e}")

        except Exception as e:
            print(f"'{keyword}' 더 이상 '더 보기' 버튼이 없습니다. 기사 로드를 완료합니다.")
            break

    # 각 키워드 검색 후 기사 수 출력
    print(f"'{keyword}'에 대한 기사 수: {len(results)}")
    
    return results

# 결과를 엑셀로 저장하는 함수
def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)
    print(f"Total number of articles: {len(df)}")

all_results = []
# 각 키워드에 대해 검색하고, 결과를 articles에 추가
for keyword in keywords:
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)

# 전체 기사 수 출력
print(f"전체 기사 수: {len(all_results)}")

# 결과를 엑셀 파일로 저장
save_to_excel(all_results, f'3매일경제 {start_date}~{end_date}.xlsx')

driver.quit()


In [ ]:
# 본문 추출

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 서브 타이틀을 스크랩하는 함수 정의
def get_content_and_subtitle_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 본문을 담고 있는 두 가지 가능성 있는 div 태그
        article_body = soup.find('div', class_='news_cnt_detail_wrap') or soup.find('div', class_='article_body')

        # 서브 타이틀 추출
        subtitle = ""
        subtitle_tag = soup.find('div', class_='midtitle_text')
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True)
            subtitle_tag.decompose()  # 서브 타이틀 본문에서 제거

        if article_body:
            # 'caption' 클래스를 가진 모든 태그를 제거합니다.
            for caption in article_body.find_all('div', class_='thumb_area'):
                caption.decompose()

            # 광고 제거
            ad = article_body.find('a', target='_blank')
            if ad:
                next_sibling = ad.find_next_sibling()
                while next_sibling:
                    next_sibling.decompose()
                    next_sibling = ad.find_next_sibling()
                ad.decompose()  # ad 자체도 제거    

            content = article_body.get_text(separator='\n', strip=True)

        else:
            content = ""

        return content, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문과 서브 타이틀을 스크랩하여 content와 sub-title 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_and_subtitle_from_url(row['url'])
    
    if content is None:
        error_log.append(row['url'])
    else:
        df.at[index, 'content'] = content
        df.at[index, 'sub-title'] = subtitle

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 오류 로그 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문과 서브 타이틀을 성공적으로 스크랩하여 저장했습니다.")
